# Electronic Add-Ons for Product Sales Optimization - Dataset Validation
Created by  : Ediashta Narendra - ver1.1 10 Apr 2024
<br>
Dataset     : Electronic Transaction Dataset - cleaned / electronic_data_clean.csv

## Introduction

Created by: **Ediashta Narendra**

This project is designed to automate the process of extracting electronic product transaction data from PostgreSQL, performing data cleaning in Python, and exporting the processed dataset into Elasticsearch for further analysis and visualization using Kibana. The pipeline is orchestrated using Apache Airflow and containerized with Docker for portability and scalability.

The dataset includes electronic transaction records from **2023 to 2024**, consisting of approximately **20,000 rows**. The raw dataset (`electronic_data_raw.csv`) and the cleaned dataset (`electronic_data_clean.csv`) are both included in this repository.


## Validation Objectives

To ensure the reliability and quality of the cleaned dataset, **Great Expectations** is integrated into the pipeline. The validation covers multiple data integrity and quality checks, including:

- **Data Consistency**:
  - `expect_column_values_to_be_in_set`: Ensures values are limited to valid sets (e.g., `gender`, `payment_method`, `rating`).
  - `expect_column_values_to_match_regex`: Enforces consistent ID formats (e.g., `transaction_id` follows a strict pattern like `^TRS\d{6}\d{4}$`).

- **Uniqueness and Completeness**:
  - `expect_column_values_to_be_unique`: Guarantees that transaction IDs are unique.
  - `expect_column_value_lengths_to_equal`: Validates ID length (e.g., `transaction_id` must be 13 characters long).

- **Type and Range Validation**:
  - `expect_column_values_to_be_in_type_list`: Ensures fields like `customer_id` are of the correct type.
  - `expect_column_min_to_be_between` and `expect_column_max_to_be_between`: Validate that values such as `age` fall within acceptable ranges.
  - `expect_column_pair_values_a_to_be_greater_than_b`: Confirms logical relationships (e.g., `total_price` ≥ `unit_price`).

These validations help detect anomalies early, prevent downstream errors, and maintain high data quality throughout the pipeline.


## Import & Install Libraries

In [26]:
# Install the gx
!pip install -q "great-expectations==0.18.19"

In [27]:
pip install --upgrade pydantic

Note: you may need to restart the kernel to use updated packages.


In [28]:
#import library
from great_expectations.data_context import FileDataContext

## Create Data Context


In [29]:
# Create a data context
context = FileDataContext.create(project_root_dir='./')

## Connect to a Datasource

In [30]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'cleancsv-electronic-transaction'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'electronic-transaction-clean'
path_to_data = 'electronic_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

## Create Expectation Suite

In [31]:
# Creat an expectation suite
expectation_suite_name = 'expectation-electronic-transaction'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,transaction_id,customer_id,age,gender,loyalty_member,product_type,sku,rating,order_status,payment_method,total_price,unit_price,quantity,purchase_date,shipping_type,add-on_total,add_on_1,add_on_2,add_on_3
0,TRS2409230001,4914,74,Male,No,Smartwatch,SKU1003,3,Completed,Debit Card,3379.32,844.83,4,2023-09-24,Overnight,106.12,Impulse Item,Impulse Item,Extended Warranty
1,TRS2409230002,1035,32,Female,No,Smartphone,SKU1001,5,Completed,Cash,186.75,20.75,9,2023-09-24,Standard,0.00,NaN,NaN,NaN
2,TRS2409230003,4702,46,Female,No,Smartwatch,SKU1003,3,Cancelled,PayPal,1689.66,844.83,2,2023-09-24,Express,86.55,Extended Warranty,Extended Warranty,Impulse Item
3,TRS2409230004,1419,55,Female,Yes,Smartphone,SKU1001,5,Completed,Credit Card,124.50,20.75,6,2023-09-24,Overnight,76.59,Accessory,Accessory,Accessory
4,TRS2409230005,9016,61,Female,No,Smartphone,SKU1001,5,Completed,PayPal,83.00,20.75,4,2023-09-24,Express,107.39,Impulse Item,Extended Warranty,Extended Warranty


## Expectation 1 - To Be Unique

In [32]:
# Expectation 1 : Column `transaction_id` must be unique
validator.expect_column_values_to_be_unique('transaction_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 2 - To Be Between min_value and max_value

In [33]:
# Expectation 2 : Column `age` must be between max value
validator.expect_column_max_to_be_between(column="age", min_value=17,max_value=80)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 80
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [34]:
# Expectation 3 : Column `age` must be between min value
validator.expect_column_min_to_be_between(column="age", min_value=17, max_value=80)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 18
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 3 - To Be In Set

In [35]:
# Expectation 4 : Column `rating` must be in set [1, 2, 3, 4, 5]
validator.expect_column_values_to_be_in_set(column="rating",value_set=[1, 2, 3, 4, 5])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [36]:
# Expectation 5 : Column `gender` must be in set ['Male','Female']
validator.expect_column_values_to_be_in_set('gender', ['Male','Female'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [37]:
# Expectation 6 : Column `gender` must be in set ['Credit Card','Bank Transfer','PayPal','Cash','Debit Card']
validator.expect_column_values_to_be_in_set('payment_method', ['Credit Card','Bank Transfer','PayPal','Cash','Debit Card'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 4 - To Be In Type List

In [38]:
# Expectation 7 : Column `customer_id` must be in type list INTEGER
validator.expect_column_values_to_be_in_type_list(column="customer_id", type_list=["INT"])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 5 - To be Not Null

In [39]:
# Expectation 8 : Column `transaction_id` can not contain missing values
validator.expect_column_values_to_not_be_null('transaction_id')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [40]:
# Expectation 9 : Column `purchase_date` can not contain missing values
validator.expect_column_values_to_not_be_null('purchase_date')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 6 - To Be Match with Regex

In [41]:
# Expectation 10 : Column `transaction_id` must be match with regex "TRS\d{6}\d{4}$"
validator.expect_column_values_to_match_regex(column="transaction_id",regex="^TRS\d{6}\d{4}$")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 7 - To Be Greater Than

In [42]:
## Expectation 11 - column 'total_price' must be greater than 'unit_price'
validator.expect_column_pair_values_a_to_be_greater_than_b(column_A="total_price",column_B="unit_price",or_equal=True)


Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Expectation 8 - Column Value Lenghts to Be Equal 

In [43]:
## Expectation 12 - column 'transaction_id' must be equal to 13
validator.expect_column_value_lengths_to_equal(column="transaction_id",value=13)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 20000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}